In [1]:
from glob import glob
from os.path import join
import pandas as pd

from maps.pings import Pings
from maps.maps import FoliumMap

location_data_dirpath = '../../data/locations/'
photo_metadata_dirpath = '../../data/photo_metadata'

pings_data_path = join(location_data_dirpath, 'pings.hdf')
posts_path = join(photo_metadata_dirpath, 'posts.hdf')

%reload_ext autoreload
%autoreload 2

In [886]:
# load posts and all pings
ping_data = pd.read_hdf(pings_data_path, 'data')
ping_data = ping_data[ping_data.accuracy < 3000]
ping_data = ping_data[ping_data.altitude < 4000]

posts = pd.read_hdf(posts_path, 'data')

In [887]:
# geocode trip pings
pings = Pings(ping_data.loc['2019-07-24':], 2)
pings.cluster(use_weights=False, n_clusters=500)
pings.build_geocodes()
pings.geocode()

In [904]:
m = FoliumMap()
m.build_map(width='70%', height='70%')
m.add_bubbles(pings.centroids, pings.geocodes.index.values)
m.map

In [1052]:
from folium import FeatureGroup, Icon, Marker, PolyLine, TileLayer
from folium.features import CustomIcon
from folium.plugins import MarkerCluster, HeatMap, AntPath
from maps.popup import ImagePopup

class TripSegment:
    
    GPS_INDEX = ['latitude', 'longitude']
    
    def __init__(self, data, owner=None):
        self.data = data
        self.owner = owner
        
    @property
    def start(self):
        return self.data.index.values[0]
    
    @property
    def stop(self):
        return self.data.index.values[1]
    
    @property
    def dt(self):
        return self.stop-self.start
    
    @property
    def layer_id(self):
        for layer_id in self.data.trip_id.unique():
            if type(layer_id)==str:
                break
        return layer_id
    
    @property
    def origin(self):
        return self.data.iloc[0]
        
    @property
    def destination(self):
        return self.data.iloc[-1]
    
    @property
    def international(self):
        return self.origin.country != self.destination.country
    
    @staticmethod
    def fmt_location(location):
        
        if location.country == 'US':
            return '{:s}, {:s}'.format(location.state, location.country)
        else:
            return '{:s}'.format(location.country)
    
    @property
    def caption(self):
        origin_str = self.fmt_location(self.origin)
        destination_str = self.fmt_location(self.destination)
        caption = '{:s} --> {:s}'.format(origin_str, destination_str)
        return caption
        
    def get_line(self, **kwargs):
        xy = self.data[self.GPS_INDEX].values
        return PolyLine(xy, tooltip=self.caption, **kwargs)
                
    def get_antpath(self, **kwargs):
        xy = self.data[self.GPS_INDEX].values
        return AntPath(xy, tooltip=self.caption, **kwargs)
    
    def get_heatmap(self, radius=7, blur=7, **kwargs):
        xy = self.data[self.GPS_INDEX].values
        heatmap = HeatMap(xy,
                          radius=radius,
                          blur=blur,
                          **kwargs)
        
        return heatmap
        
            
class FlightSegment(TripSegment):
    pass

class DriveSegment(TripSegment):
    pass
        
class TrainSegment(TripSegment):
    pass

In [1053]:
from modules.utilities import haversine

def find_flights(pings, owner=None):
    SEMANTIC_GPS_INDEX = ['latitude_geocode', 'longitude_geocode']
    
    flights = []
    pings = pings.sort_index()    
    gps = pings[SEMANTIC_GPS_INDEX].values.astype(float)
    dx = np.array([haversine(*p) for p in zip(gps[:-1], gps[1:])])
    dt = np.array((pings.index.values[1:] - pings.index.values[:-1]).tolist()) / 1e9 / 3600 # hours
    for idx in np.logical_and(dx>250, dx/dt>25).nonzero()[0]:
        flight = FlightSegment(pings.iloc[[idx, idx+1]], owner=owner)
        flights.append(flight)    
    flights = [flight for flight in flights if flight.international or flight.origin.country=='US']
    return flights

def find_drives(pings, transits):
    transits = sorted(transits, key=lambda x: x.start)
    all_pings = pings.pings.droplevel(0).sort_index()
    drives = [DriveSegment(all_pings[: transits[0].start])]
    for i, transit in enumerate(transits[:-1]):
        drives.append(DriveSegment(all_pings[transit.stop: transits[i+1].start]))
    drives.append(DriveSegment(all_pings[transits[-1].stop:]))
    return drives

In [ ]:
from sklearn.cluster import MeanShift

# cluster photos
photos = posts[~posts.latitude.isna()]
photos.loc[:, 'gallery'] = None
for idx, album in photos.groupby('album'):
    model = MeanShift(bandwidth=1.).fit(album[Pings.GPS_INDEX].values)
    gallery_base = '-'.join([x.lower() for x in idx.split()])
    labels = [gallery_base+'{:d}'.format(_id) for _id in model.labels_]
    photos.loc[album.index, 'gallery'] = labels

In [1125]:
# add trip ids
trip_ids = {
    'July': ('2019-07-24', '2019-07-28'),
    'August-November': ('2019-08-11', '2019-11-09'),
    'December': ('2019-12-01', '2019-12-24'),
    'February': ('2020-02-25', '2020-03-17')
}

pings._pings['trip_id'] = None
for trip_id, dates in trip_ids.items():
    after_start = pings._pings.index.get_level_values(1) >= dates[0]
    before_end = pings._pings.index.get_level_values(1) <= dates[1]
    pings._pings.loc[after_start&before_end, 'trip_id'] = trip_id

In [1126]:
# find trains/planes/drives
trains = [
    TrainSegment(pings.pings.loc['SMB'].loc['2019-08-15 22:31:30':'2019-08-16 07:23:57']),
    TrainSegment(pings.pings.loc['CMB'].loc['2019-09-02 04:43:04':'2019-09-02 06:03:54'])]
flights = find_flights(pings.pings.loc['SMB'])
drives = find_drives(pings, trains+flights)

In [1140]:
m = FoliumMap()
m.build_map(width='100%', height='100%')
TileLayer('openstreetmap').add_to(m.map)
TileLayer('Stamen Terrain').add_to(m.map)

layers = {name: FeatureGroup(name=name, show=False).add_to(m.map) for name in itineraries.keys()}
fg_heatmap = FeatureGroup(name='Heatmap', show=True).add_to(m.map)
fg_photos = FeatureGroup(name='Photos', show=True).add_to(m.map)

for flight in flights:
    obj = flight.get_line(color='black', weight=3, opacity=0.2)
    obj.add_to(layers[flight.layer_id])
    
for train in trains:
    obj = train.get_line(color='blue', weight=3, opacity=0.2)
    obj.add_to(layers[train.layer_id])
    
for drive in drives:
    hm = drive.get_heatmap()
    hm.add_to(fg_heatmap)
    obj = drive.get_antpath(color='red', weight=3)
    obj.add_to(layers[drive.layer_id])
    
    
# photo clusters
mc = MarkerCluster().add_to(fg_photos)
for idx, photo in photos.iterrows():
    xy = photo[Pings.GPS_INDEX].values.astype(float)
    popup = ImagePopup(photo.imgur_id, photo.caption, photo.gallery).popup
    tooltip = photo.caption
    Marker(xy, popup=popup, 
           tooltip=tooltip, 
           icon=Icon('darkred', icon_color='white', icon='image', prefix='fa')
          ).add_to(mc)
        
# add the layer control
LayerControl().add_to(m.map)

In [1143]:
m.map.save('../travel/map.html')

In [1071]:
# geocode trip pings
photo_pings = Pings(ping_data.loc['2019-07-24':], 1)
photo_pings.cluster(use_weights=False, n_clusters=30)

In [1064]:
photo_pings.model.predict()

<bound method Pings.cluster of <maps.pings.Pings object at 0x1ae449470>>

In [1072]:
m = FoliumMap()
m.build_map()
m.add_bubbles(photo_pings.centroids)
m.map

In [1061]:
photos.al

path  \
album           filename     source                                                         
CA Highway 1    IMG_9558.jpg iPhone SE  /Volumes/bernasek_t5/photos/lightroom/rendered...   
                IMG_9562.jpg iPhone SE  /Volumes/bernasek_t5/photos/lightroom/rendered...   
                IMG_9569.jpg iPhone SE  /Volumes/bernasek_t5/photos/lightroom/rendered...   
                IMG_9715.jpg iPhone SE  /Volumes/bernasek_t5/photos/lightroom/rendered...   
                IMG_9580.jpg iPhone SE  /Volumes/bernasek_t5/photos/lightroom/rendered...   
...                                                                                   ...   
Rocky Mountains IMG_9729.jpg iPhone 7   /Volumes/bernasek_t5/photos/lightroom/rendered...   
                IMG_9731.jpg iPhone 7   /Volumes/bernasek_t5/photos/lightroom/rendered...   
                IMG_5128.jpg iPhone SE  /Volumes/bernasek_t5/photos/lightroom/rendered...   
                IMG_5131.jpg iPhone SE  /Volumes/bernasek_t5/photos/lightroom/rendered...   
                IMG_9741.jpg iPhone 7   /Volumes/bernasek_t5/photos/lightroom/rendered...   

                                                 timestamp  \
album           filename     source                          
CA Highway 1    IMG_9558.jpg iPhone SE 2019-07-24 18:20:35   
                IMG_9562.jpg iPhone SE 2019-07-24 18:50:17   
                IMG_9569.jpg iPhone SE 2019-07-24 19:38:56   
                IMG_9715.jpg iPhone SE 2019-07-25 12:27:55   
                IMG_9580.jpg iPhone SE 2019-07-25 12:31:10   
...                                                    ...   
Rocky Mountains IMG_9729.jpg iPhone 7  2020-03-16 10:42:31   
                IMG_9731.jpg iPhone 7  2020-03-16 10:44:10   
                IMG_5128.jpg iPhone SE 2020-03-16 14:10:45   
                IMG_5131.jpg iPhone SE 2020-03-16 14:16:06   
                IMG_9741.jpg iPhone 7  2020-03-16 17:37:14   

                                              time_shot_pst  \
album           filename     source                           
CA Highway 1    IMG_9558.jpg iPhone SE  2019:07:24 18:20:35   
                IMG_9562.jpg iPhone SE  2019:07:24 18:50:17   
                IMG_9569.jpg iPhone SE  2019:07:24 19:38:56   
                IMG_9715.jpg iPhone SE  2019:07:25 12:27:55   
                IMG_9580.jpg iPhone SE  2019:07:25 12:31:10   
...                                                     ...   
Rocky Mountains IMG_9729.jpg iPhone 7   2020:03:16 10:42:31   
                IMG_9731.jpg iPhone 7   2020:03:16 10:44:10   
                IMG_5128.jpg iPhone SE  2020:03:16 14:10:45   
                IMG_5131.jpg iPhone SE  2020:03:16 14:16:06   
                IMG_9741.jpg iPhone 7   2020:03:16 17:37:14   

                                            time_shot_local  \
album           filename     source                           
CA Highway 1    IMG_9558.jpg iPhone SE  2019:07:24 18:20:35   
                IMG_9562.jpg iPhone SE  2019:07:24 18:50:17   
                IMG_9569.jpg iPhone SE  2019:07:24 19:38:56   
                IMG_9715.jpg iPhone SE  2019:07:25 12:27:55   
                IMG_9580.jpg iPhone SE  2019:07:25 12:31:10   
...                                                     ...   
Rocky Mountains IMG_9729.jpg iPhone 7   2020:03:16 11:42:31   
                IMG_9731.jpg iPhone 7   2020:03:16 11:44:10   
                IMG_5128.jpg iPhone SE  2020:03:16 15:10:45   
                IMG_5131.jpg iPhone SE  2020:03:16 15:16:06   
                IMG_9741.jpg iPhone 7   2020:03:16 18:37:14   

                                              time_rendered geotagged  \
album           filename     source                                     
CA Highway 1    IMG_9558.jpg iPhone SE  2020:04:09 12:51:46      True   
                IMG_9562.jpg iPhone SE  2020:04:09 12:51:46      True   
                IMG_9569.jpg iPhone SE  2020:04:09 12:51:48      True   
                IMG_9715.jpg iPhone SE  2020:04:09 12:51:48    

In [1036]:
 
    location=[46.3014, -123.7390],
    zoom_start=7,
    tiles='Stamen Terrain'
)

folium.Marker(
    location=[47.3489, -124.708],
    popup=folium.Popup(max_width=450),
    icon=folium.Icon(color='red', icon='image', prefix='fa')).add_to(m)

m

In [981]:
photo.caption

'Hansen, ID'

In [ ]:
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=df_counters['Name'][point], 
                  icon=folium.Icon(color='darkblue', icon_color='white', icon='male', 
                                   angle=0, prefix='fa')).add_to(marker_cluster)

In [958]:
m = FoliumMap()
m.build_map(width='70%', height='70%')
m.add_bubbles(photo_gps, color='blue', fill_color='blue', radius=5)
m.map

In [7]:
m.save('./TEST_MAP.html')